In [42]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import dependencies

In [4]:
#from selenium import webdriver
import os
import shlex
import subprocess

import requests
from bs4 import BeautifulSoup

## Get the iframe url of any episode url

In [5]:
def get_iframe_src(url):
  print("Getting iframe url")
  response = requests.get(url)

  soup = BeautifulSoup(response.text, features="lxml")
  player_list_block = soup.find("div", class_="filmicerik")
  players = player_list_block.find_all("iframe")

  for player in players:
    if "https://ok.ru/videoembed/" in player["src"]:
      return player["src"]


## Get episode urls of the drama

In [6]:
def fetch_episodes(drama_url):
  print("Fetching episode urls....")

  try:
    response = requests.get(drama_url)
  except Exception as error:
    print(f"Failed to get the url. {error}")
    return None
  else:
    if response.status_code == 200:
      episodes = [
          {
              "episode": "1",
              "url": drama_url 
          } 
      ]
      try:
        soup = BeautifulSoup(response.text, features="lxml")
        for a in soup.find("div", class_="keremiya_part").find_all("a"):
          episode = a["href"].split("/")[-2]
          episode_url = a["href"]

          episodes.append(
              {
                  "episode": episode,
                  "url": episode_url
              }
          )
      except Exception as error:
        print(f"Failed to scrape the webpage. {error}")
        return None
      else:
        return episodes


## Download one episode by its iframe url (from ok.ru)

In [7]:
def download_episode(episode_url):
  command = f"yt-dlp '{episode_url}' --verbose"

  process = None
  try:
      process = subprocess.Popen(
          shlex.split(command), shell=False, stdout=subprocess.PIPE)
  except Exception as error:
    print(f"An error occurred when downloading episode. {error}")
  else:
    while True:
        output = process.stdout.readline()
        # used to check for empty output in Python2, but seems
        # to work with just poll in 2.7.12 and 3.5.2
        # if output == '' and process.poll() is not None:
        if process.poll() is not None:
            break
        if output:
            print(output.strip().decode())
    rc = process.poll()
    print("Episode downloaded successfully.")

## Main program

In [11]:
def main():
  not_finished = True
  print("-----------KDRAMA-VOSTFR-DOWNLOADER------------")
  while not_finished:
    URL = input("Please provide the drama url : ")

    episodes = fetch_episodes(URL)
    if episodes is not None:
      print(f"Fetched episodes. Number of episodes of this drama is {len(episodes)}.")

      print("-------------------EPISODES--------------------")
      for episode in episodes:
        print(f"Episode {episode.get('episode')}: {episode.get('url')}")

      print("\n-------------AVAILABLE-OPTIONS----------------")
      print("[1] : Download all episodes")
      print("[2] : Download one episode")
      print("[Q] : Quit")
      while True:
        option = input("What do you want to do ? ")

        if option not in ("1", "2", "Q"):
          print("Wrong command. Please type either 1, 2, or Q.")
        else:
          if option == "1":
            print("Downloading all episodes....")
            for episode in episodes:
              print(f"\nDownloading episode {episode.get('episode')}....")
              iframe_src = get_iframe_src(episode.get("url"))
              download_episode(iframe_src)
            print("DONE downloading all episodes.")
            not_finished = False
            break

          elif option == "2":
            while True:
              episode = input("Please provide the episode that you want to download : ")
              if episode not in (d["episode"] for d in episodes):
                print("This episode doesn't exist. Please provide a valid episode.")
              else:
                print(f"\nDownloading episode {episode}....")
                for ep in episodes:
                  if ep.get('episode') == episode:
                    iframe_src = get_iframe_src(ep.get("url"))
                    download_episode(iframe_src)
                    break
                print("DONE.")
                not_finished = False
                break

          elif option == "Q":
            print("Quitting program.")
            not_finished = False
            break


In [13]:
main()

-----------KDRAMA-VOSTFR-DOWNLOADER------------
Fetching episode urls....
Fetched episodes. Number of episodes of this drama is 20.
------------EPISODES--------------
Episode 1: https://kdramavostfr.com/mouse-vostfr-kdrama/
Episode 2: https://kdramavostfr.com/mouse-vostfr-kdrama/2/
Episode 3: https://kdramavostfr.com/mouse-vostfr-kdrama/3/
Episode 4: https://kdramavostfr.com/mouse-vostfr-kdrama/4/
Episode 5: https://kdramavostfr.com/mouse-vostfr-kdrama/5/
Episode 6: https://kdramavostfr.com/mouse-vostfr-kdrama/6/
Episode 7: https://kdramavostfr.com/mouse-vostfr-kdrama/7/
Episode 8: https://kdramavostfr.com/mouse-vostfr-kdrama/8/
Episode 9: https://kdramavostfr.com/mouse-vostfr-kdrama/9/
Episode 10: https://kdramavostfr.com/mouse-vostfr-kdrama/10/
Episode 11: https://kdramavostfr.com/mouse-vostfr-kdrama/11/
Episode 12: https://kdramavostfr.com/mouse-vostfr-kdrama/12/
Episode 13: https://kdramavostfr.com/mouse-vostfr-kdrama/13/
Episode 14: https://kdramavostfr.com/mouse-vostfr-kdrama/14/

[debug] Command-line config: ['https://ok.ru/videoembed/3435104635411', '--verbose']
[debug] Encodings: locale UTF-8, fs utf-8, out utf-8 (No ANSI), err utf-8 (No ANSI), pref UTF-8
[debug] yt-dlp version 2022.04.08 [7884ade65]
[debug] Python version 3.10.2 (CPython 64bit) - Linux-5.10.96-1-MANJARO-x86_64-with-glibc2.33
[debug] Checking exe version: ffmpeg -bsfs
[debug] Checking exe version: ffprobe -bsfs
[debug] exe versions: ffmpeg 4.4.1 (setts), ffprobe 4.4.1, rtmpdump 2.4
[debug] Optional libraries: certifi, Cryptodome, secretstorage, mutagen, sqlite, websockets
[debug] Proxy map: {}
[debug] [Odnoklassniki] Extracting URL: https://ok.ru/videoembed/3435104635411


[Odnoklassniki] 3435104635411: Downloading desktop webpage
[Odnoklassniki] 3435104635411: Downloading m3u8 information


[debug] Formats sorted by: hasvid, ie_pref, lang, quality, res, fps, hdr:12(7), vcodec:vp9.2(10), acodec, filesize, fs_approx, tbr, vbr, abr, asr, proto, vext, aext, hasaud, source, id
[debug] Default format spec: bestvideo*+bestaudio/best
[debug] Invoking downloader on "https://vd343.mycdn.me/?expires=1650270565658&srcIp=102.16.43.210&pr=10&srcAg=CHROME&ms=185.226.53.59&type=3&sig=6Epz5Yac0D8&ct=4&urls=45.136.22.39&clientType=0&id=1818717260307"


[info] 3435104635411: Downloading 1 format(s): mpd-4
[download] Destination: Mouse E16 Fr [3435104635411].mp4


KeyboardInterrupt: 

In [ ]:
!zip -r -9 kingdom-s01.zip Kingdom-S01/